In [1]:
from gensim.models import KeyedVectors

w2v_model = KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin',
                                              binary=True)

c:\users\jiayee\documents\gitrepositories\pms\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import numpy as np

w2v_vocab = len(w2v_model.vocab)
w2v_dim = 300

In [3]:
window_size = 2
phrase = 2 * window_size + 1

batch_size = 20
epochs = 1

In [4]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense

input_length = phrase
input_shape = (phrase, w2v_dim)

dropout_probability = 0.2

layers = [
    LSTM(phrase,
         input_shape=input_shape,
         dropout=dropout_probability,
         recurrent_dropout=dropout_probability,
         return_sequences=True),
    LSTM(phrase,
         dropout=dropout_probability,
         recurrent_dropout=dropout_probability,
         input_shape=input_shape),
    Dense(1,
          activation='softmax')
]

def generate_model():
    model = Sequential()
    for layer in layers:
        model.add(layer)
    model.compile(optimizer='adam',
                  metrics=['accuracy'],
                  loss='categorical_crossentropy')
    return model

def fit_model(model, X, Y):
    model.fit(X,
              Y,
              batch_size=batch_size,
              epochs=epochs)
    return model

def save_model_to_file(model, file_name):
    model.save('models/{}.h5'.format(file_name))

c:\users\jiayee\documents\gitrepositories\pms\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
from keras.models import Sequential, load_model

def load_model_from_file(file_name):
    model = load_model('models/{}.h5'.format(file_name))
    return model
    
def evaluate_model(model, X, Y):
    score = model.evaluate(X,
                           Y,
                           batch_size=batch_size)
    return score

In [6]:
def get_lemma(lexelt):
    return lexelt.split('.')[0]

def get_pos(lexelt):
    return lexelt.split('.')[1]

def is_match_lexelt(synset, lexelt_item):
    lexelt_with_sense = synset.name()
    lemma_a, pos_a = get_lemma(lexelt_with_sense), get_pos(lexelt_with_sense)
    lemma_b, pos_b = get_lemma(lexelt_item), get_pos(lexelt_item)
    return (lemma_a.lower() == lemma_b.lower() and
            pos_a.lower() == pos_b.lower())

In [7]:
from keras.utils.np_utils import to_categorical
from lxml import etree
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize, word_tokenize
from pickle import dump

train_or_test = 'train'
file = 'data/semeval2007/{0}/lexical-sample/english-lexical-sample.{0}.xml'.format(train_or_test)
root = etree.parse(file)

trained_lexelt_items = set()
instance_indexer = {} # { instance_id: X.index(instance_id) and Y.index(instance_id) }
lexelt_label_indexer = {} # { lexelt.item: { answer.sense_id: one-hot index } }

for lexelt in root.findall('lexelt'):
    lexelt_item = lexelt.attrib['item']
    lexelt_pos = lexelt.attrib['pos']
    instances = lexelt.findall('instance')

    number_of_instances = len(instances)
    if number_of_instances not in range(50, 100):
        continue

    number_of_classes = len(
        list(filter(
            lambda synset: is_match_lexelt(synset, lexelt_item),
            wordnet.synsets(get_lemma(lexelt_item))
        ))
    ) + 1 # because sense id begins from 1

    print(lexelt_item, number_of_classes)

    X = np.zeros((number_of_instances, phrase, w2v_dim), dtype=np.float64)
    Y = np.zeros((number_of_instances, number_of_classes), dtype=np.uint8)

    for instance_index, instance in enumerate(instances):
        instance_id = instance.attrib['id']
        instance_indexer[instance_id] = instance_index

        answer_sense_id = instance.find('answer').attrib['senseid']
        if lexelt_item not in lexelt_label_indexer:
            lexelt_label_indexer[lexelt_item] = {}
        if answer_sense_id not in lexelt_label_indexer[lexelt_item]:
            lexelt_label_indexer[lexelt_item][answer_sense_id] = len(lexelt_label_indexer[lexelt_item]) + 1 # because sense id
        try:
            label_index = lexelt_label_indexer[lexelt_item][answer_sense_id]
            Y[instance_index] = to_categorical(label_index, num_classes=number_of_classes)
        except: # IndexError because number of senses for the lexelt in WordNet 3.0 < number of senses in WordNet 1.7 or 2.1
            lexelt_label_indexer.pop(lexelt_item, None)
            break

        context = instance.find('context')
        head = context.find('head').text.strip()
        etree.strip_tags(context, 'head')
        words = list(map(lambda sentence: word_tokenize(sentence), sent_tokenize(context.text)))
        sentence_index, word_index = -1, -1
        for (s_index, sentence) in enumerate(words):
            for (w_index, word) in enumerate(sentence):
                if word == head:
                    sentence_index, word_index = s_index, w_index
                    break
        if sentence_index == -1 or word_index == -1: # Lexelt did not exist in the context
            continue

        sentence = words[sentence_index]
        lower_bound = max(0, word_index - window_size)
        upper_bound = min(word_index + window_size, len(sentence))
        w2v_vectors = np.empty((phrase, w2v_dim))
        for w_index in range(lower_bound, upper_bound):
            word = sentence[w_index]
            if word in w2v_model:
                w2v_vectors[w_index - lower_bound] = w2v_model[word] # Set to ref to this vector
        X[instance_index] = w2v_vectors

    if lexelt_item not in lexelt_label_indexer:
        continue

    layers[-1] = Dense(number_of_classes,
                      activation='softmax')
    model = generate_model()
    fit_model(model, X, Y)
    save_model_to_file(model, lexelt_item)
    trained_lexelt_items.add(lexelt_item)
    print(model.summary())
    print()

with open('lexelts.bin', 'wb') as l:
    dump(trained_lexelt_items, l)

with open('sense_id_indexer.bin', 'wb') as s:
    dump(lexelt_label_indexer, s)

announce.v 5
Epoch 1/1
88/88 [==============================] - 5s 62ms/step - loss: nan - acc: 0.0682
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 5, 5)              6120      
_________________________________________________________________
lstm_2 (LSTM)                (None, 5)                 220       
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 30        
Total params: 6,370
Trainable params: 6,370
Non-trainable params: 0
_________________________________________________________________
None

approve.v 3
Epoch 1/1
53/53 [==============================] - 6s 119ms/step - loss: nan - acc: 0.0000e+00
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 5, 5)              6

Epoch 1/1
52/52 [==============================] - 11s 216ms/step - loss: nan - acc: 0.0000e+00
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 5, 5)              6120      
_________________________________________________________________
lstm_2 (LSTM)                (None, 5)                 220       
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 18        
Total params: 6,358
Trainable params: 6,358
Non-trainable params: 0
_________________________________________________________________
None

explain.v 3
Epoch 1/1
85/85 [==============================] - 12s 144ms/step - loss: nan - acc: 0.0000e+00
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 5, 5)              6120   

In [ ]:
from keras.utils.np_utils import to_categorical
from lxml import etree
from nltk.tokenize import sent_tokenize, word_tokenize
from pickle import load

train_or_test = 'test'
file = 'data/semeval2007/{0}/lexical-sample/english-lexical-sample.{0}.xml'.format(train_or_test)
root = etree.parse(file)

with open('lexelts.bin', 'rb') as l:
    trained_lexelt_items = load(l)

with open('sense_id_indexer.bin', 'rb') as s:
    lexelt_label_indexer = load(s)

answers = {}
with open('data/semeval2007/key/english-lexical-sample.{}.key'.format(train_or_test)) as k:
    for line in k:
        lexelt_item, instance_id, answer_sense_id = line.strip().split(' ')
        if instance_id not in instance_indexer:
            continue
        instance_index = instance_indexer[instance_id]
        answers[instance_id] = answer_sense_id

instance_indexer = {} # { instance_id: X.index(instance_id) and Y.index(instance_id) }

for lexelt in root.findall('lexelt'):
    lexelt_item = lexelt.attrib['item']
    if lexelt_item not in trained_lexelt_items:
        continue
    lexelt_pos = lexelt.attrib['pos']
    instances = lexelt.findall('instance')

    number_of_instances = len(instances)
    if number_of_instances not in range(50, 100):
        continue

    number_of_classes = len(lexelt_label_indexer[lexelt_item]) + 1 # because sense id begins from 1

    X = np.zeros((number_of_instances, phrase, w2v_dim), dtype=np.float64)
    Y = np.zeros((number_of_instances, number_of_classes), dtype=np.uint8)

    for instance_index, instance in enumerate(instances):
        instance_id = instance.attrib['id']
        instance_indexer[instance_id] = instance_index

        context = instance.find('context')
        head = context.find('head').text.strip()
        etree.strip_tags(context, 'head')
        words = list(map(lambda sentence: word_tokenize(sentence), sent_tokenize(context.text)))
        sentence_index, word_index = -1, -1
        for (s_index, sentence) in enumerate(words):
            for (w_index, word) in enumerate(sentence):
                if word == head:
                    sentence_index, word_index = s_index, w_index
                    break
        if sentence_index == -1 or word_index == -1: # Lexelt did not exist in the context
            continue

        sentence = words[sentence_index]
        lower_bound = max(0, word_index - window_size)
        upper_bound = min(word_index + window_size, len(sentence))
        w2v_vectors = np.empty((phrase, w2v_dim))
        for w_index in range(lower_bound, upper_bound):
            word = sentence[w_index]
            if word in w2v_model:
                w2v_vectors[w_index - lower_bound] = w2v_model[word] # Set to ref to this vector
        X[instance_index] = w2v_vectors

        answer_sense_id = answers[instance_id]
        label_index = lexelt_label_indexer[lexelt_item][answer_sense_id]
        Y[instance_index] = to_categorical(label_index, num_classes=number_of_classes)

    model = load_model_from_file(lexelt_item)
    score = evaluate_model(model, X, Y)
    print(lexelt_item)
    print(score)
    print()